# 使用飞桨构建波士顿房价预测模型

<br></br>
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/1c9a9b3b90b44dfdbba227098db93192aa52730d8fe647faad7c9fca1912e2c9" width="800" hegiht="" ></center>
<center><br>图1：使用飞桨框架构建神经网络过程</br></center>
<br></br>

In [1]:
#加载飞桨、Numpy和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

代码中参数含义如下：

* paddle：飞桨的主库，paddle 根目录下保留了常用API的别名，当前包括：paddle.tensor、paddle.framework目录下的所有API。

* paddle.nn：组网相关的API，例如 Linear 、卷积 Conv2D 、 循环神经网络 LSTM 、损失函数 CrossEntropyLoss 、 激活函数 ReLU 等。

* Linear：神经网络的全连接层函数，即包含所有输入权重相加的基本神经元结构。在房价预测任务中，使用只有一层的神经网络（全连接层）来实现线性回归模型。

* paddle.nn.functional：与paddle.nn一样，包含组网相关的API，例如Linear、激活函数ReLu等。两者下的同名模块功能相同，运行性能也基本一致。 但是，paddle.nn下的模块均是类，每个类下可以自带模块参数；paddle.nn.functional下的模块均是函数，需要手动传入模块计算需要的参数。在实际使用中，卷积、全连接层等层本身具有可学习的参数，建议使用paddle.nn模块，而激活函数、池化等操作没有可学习参数，可以考虑直接使用paddle.nn.functional下的函数代替。
<br></br>

## 1、数据处理

In [2]:
def load_data():
    # 从文件导入数据
    datafile = './work/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data


## 2、模型设计

### 2.1 单层神经网络

In [24]:
# 单层网络
class Regressor(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features=13, out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

### 2.2 双层神经网络

In [25]:
# 双层网络
class Regressor2(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor2, self).__init__()

        self.fc1 = Linear(in_features=13, out_features=10)
        # self.relu = paddle.nn.ReLU()
        self.fc2 = Linear(in_features=10, out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        relu = paddle.nn.ReLU()
        x1 = self.fc1(inputs)
        x2 = relu(x1)
        x = self.fc2(x2)
        return x

## 3、训练配置

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/96075d4df5ae4e01ac1491ebf176fa557bd122b646ba49238f65c9b38a98cab4" width="700" hegiht="" ></center>
<center><br>图2：训练配置流程示意图</br></center>
<br></br>

In [26]:
# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式
model.train()
# 加载数据
training_data, test_data = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

In [29]:
# 声明定义好的线性回归模型
model2 = Regressor2()
# 开启模型训练模式
model2.train()
# 加载数据
training_data2, test_data2 = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt2 = paddle.optimizer.SGD(learning_rate=0.01, parameters=model2.parameters())

每次内层循环都需要执行如下四个步骤，如 **图3** 所示
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/8154cf612a024a3f9144b4e31f59568ef9ad59c155b344919221d63bb9ccfcc8" width="700" hegiht="" ></center>
<center><br>图3：内循环计算过程</br></center>
<br></br>

1. 数据准备：将一个批次的数据先转换成np.array格式，再转换成paddle内置tensor格式。
1. 前向计算：将一个批次的样本数据灌入网络中，计算输出结果。
1. 计算损失函数：以前向计算结果和真实房价作为输入，通过损失函数square_error_cost API计算出损失函数值（Loss）。飞桨所有的API接口都有完整的说明和使用案例，在后续教程中我们会详细介绍API的查阅方法。
1. 反向传播：执行梯度反向传播``backward``函数，即从后到前逐层计算每一层的梯度，并根据设置的优化算法更新参数。

### 3.1 单层神经网络训练

In [28]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor形式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(house_features)
        
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播
        avg_loss.backward()
        # 最小化loss,更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()


epoch: 0, iter: 0, loss is: [0.08701988]
epoch: 0, iter: 20, loss is: [0.14960015]
epoch: 0, iter: 40, loss is: [0.12143403]
epoch: 1, iter: 0, loss is: [0.1108645]
epoch: 1, iter: 20, loss is: [0.30029768]
epoch: 1, iter: 40, loss is: [0.11219481]
epoch: 2, iter: 0, loss is: [0.11999248]
epoch: 2, iter: 20, loss is: [0.05003921]
epoch: 2, iter: 40, loss is: [0.05750383]
epoch: 3, iter: 0, loss is: [0.03257295]
epoch: 3, iter: 20, loss is: [0.1193314]
epoch: 3, iter: 40, loss is: [0.13905767]
epoch: 4, iter: 0, loss is: [0.05879126]
epoch: 4, iter: 20, loss is: [0.09454913]
epoch: 4, iter: 40, loss is: [0.07750959]
epoch: 5, iter: 0, loss is: [0.0570086]
epoch: 5, iter: 20, loss is: [0.01355288]
epoch: 5, iter: 40, loss is: [0.04134348]
epoch: 6, iter: 0, loss is: [0.15345433]
epoch: 6, iter: 20, loss is: [0.01702835]
epoch: 6, iter: 40, loss is: [0.08140115]
epoch: 7, iter: 0, loss is: [0.03475491]
epoch: 7, iter: 20, loss is: [0.10469504]
epoch: 7, iter: 40, loss is: [0.07139311]
epo

### 3.2 双层神经网络训练

In [30]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小
loss_record = []
# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data2)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data2[k:k+BATCH_SIZE] for k in range(0, len(training_data2), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor形式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model2(house_features)
        
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        if iter_id==40:
            loss_record.append(avg_loss.numpy())
        
        # 反向传播
        avg_loss.backward()
        # 最小化loss,更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()

epoch: 0, iter: 0, loss is: [0.07781583]
epoch: 0, iter: 20, loss is: [0.09158639]
epoch: 0, iter: 40, loss is: [0.01576852]
epoch: 1, iter: 0, loss is: [0.08322625]
epoch: 1, iter: 20, loss is: [0.0915486]
epoch: 1, iter: 40, loss is: [0.00874288]
epoch: 2, iter: 0, loss is: [0.0195106]
epoch: 2, iter: 20, loss is: [0.04410897]
epoch: 2, iter: 40, loss is: [0.00799616]
epoch: 3, iter: 0, loss is: [0.11158899]
epoch: 3, iter: 20, loss is: [0.01556674]
epoch: 3, iter: 40, loss is: [0.0873567]
epoch: 4, iter: 0, loss is: [0.02877882]
epoch: 4, iter: 20, loss is: [0.06469794]
epoch: 4, iter: 40, loss is: [0.08045385]
epoch: 5, iter: 0, loss is: [0.06108591]
epoch: 5, iter: 20, loss is: [0.00696137]
epoch: 5, iter: 40, loss is: [0.11873264]
epoch: 6, iter: 0, loss is: [0.03076179]
epoch: 6, iter: 20, loss is: [0.08963884]
epoch: 6, iter: 40, loss is: [0.01661858]
epoch: 7, iter: 0, loss is: [0.02496988]
epoch: 7, iter: 20, loss is: [0.05169668]
epoch: 7, iter: 40, loss is: [0.0705395]
epoc

## 4、保存并测试模型

In [31]:
# 保存模型参数，文件名为LR_model.pdparams
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [32]:
# 保存模型参数，文件名为LR_model2.pdparams
paddle.save(model2.state_dict(), 'LR_model2.pdparams')
print("模型保存成功，模型参数保存在LR_model2.pdparams中")

模型保存成功，模型参数保存在LR_model2.pdparams中


## 5、测试

### 5.1 单层神经网络模型测试

In [33]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])
    return one_data, label

In [34]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()
# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)
# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + avg_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + avg_values[-1]
print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[24.23688]], the corresponding label is 19.700000762939453


### 5.2 双层神经网络模型测试

In [35]:
def load_one_example2():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data2.shape[0])
    idx = -10
    one_data, label = test_data2[idx, :-1], test_data2[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])
    return one_data, label

In [39]:
# 参数为保存模型参数的文件地址
model_dict2 = paddle.load('LR_model2.pdparams')
model2.load_dict(model_dict2)
model2.eval()
# 参数为数据集的文件地址
one_data2, label2 = load_one_example2()
# 将数据转为动态图的variable格式 
one_data2 = paddle.to_tensor(one_data)
predict2 = model2(one_data)
# 对结果做反归一化处理
predict2 = predict2 * (max_values[-1] - min_values[-1]) + avg_values[-1]
# 对label数据做反归一化处理
label2 = label2 * (max_values[-1] - min_values[-1]) + avg_values[-1]
print("Inference result is {}, the corresponding label is {}".format(predict2.numpy(), label2))

Inference result is [[26.16931]], the corresponding label is 19.700000762939453


## 6、总结

综合实现波士顿房价预测的两种方式，我们可以发现，使用numpy的过程中，我们需要对损失函数进行求导，并使用numpy将传播的过程以及参数的更新公式全部写出来，这就导致了代码的可移植性差，一旦使用了不同的损失函数或者激活函数，则对于权重和偏置的求导公式就会发生变化，因此，这种代码比较难以维护，且容易出错。而使用深度学习框架，则可以用简单的几行代码就写清楚传播的过程，且不需要手动求导，节省了大量的时间，且代码简单容易维护。同时对比两者的结果和运行时间也可以发现，使用深度学习框架的运行时间更短。